In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline
plt.style.use('seaborn')
from sklearn.pipeline import Pipeline

In [17]:
x_test = pd.read_csv('data_trouser_dress/troudress_test_x.csv')
x_train = pd.read_csv('data_trouser_dress/troudress_train_x.csv')
y_train = pd.read_csv('data_trouser_dress/troudress_train_y.csv')
x_train_r = pd.read_csv('1.csv')

In [ ]:
# np_x_train = x_train.values
# plt.imshow(np_x_train[2,:].reshape(28,28),cmap = 'gray', vmin = 0.0, vmax = 1.0)

In [ ]:
# np_x_train_r = x_train_reverse.values
# plt.imshow(np_x_train_r[2,:].reshape(28,28),cmap = 'gray', vmin = 0.0, vmax = 1.0)

In [ ]:
# y_train.iloc[2]

In [ ]:
# x_train

In [ ]:
# s = x_train.iloc[25, :]
# s[783]

In [ ]:
# def h_inverse(x, a):
#     y = x.copy()
    
#     for i in range(x.shape[0]): # row
#         for j in range(x.shape[1]): # col
#             y.iat[i,j] = x.iloc[i, j//28*28+27-j%28]            
#     return y

In [18]:
x_train = x_train.append(x_train_r, ignore_index=True)

In [19]:
# fit on x1, transform on x2
def add_scaled_sum(x1, x2):
    def fit_sum(x):
        s = x.apply(sum, axis = 1)
    #     print(s.values)
        scaler = MinMaxScaler()
        scaler.fit(s.values.reshape(-1,1) )
        return scaler
    def transform_scale_sum(x, scaler): #transform the sums with the scaler
        s = x.apply(sum, axis = 1)
        return scaler.transform(s.values.reshape(-1,1))
    scaler = fit_sum(x1)
    x2['whiteness'] = transform_scale_sum(x2, scaler)
    return x2

In [20]:
# fit on x1, transform on x2
def add_central_sum(x1, x2):# must be done after add_scaled_sum
    def central_sum(S):# compute the sum of the lower middle line of the picture
        int_list = np.array(range(14))+14
        Sum = 0
        for i in int_list:
            Sum = Sum + S[i*28+13]+S[i*28+14]
        return Sum
    def fit_sum(x):
        s = x.apply(central_sum, axis = 1)
        
        scaler = MinMaxScaler()
        scaler.fit(s.values.reshape(-1,1) )
        return scaler
    def transform_scale_sum(x, scaler): #transform the sums with the scaler
        s = x.apply(central_sum, axis = 1)
        return scaler.transform(s.values.reshape(-1,1))
    scaler = fit_sum(x1)
    x2['central'] = transform_scale_sum(x2, scaler)
    return x2

In [21]:
x_train = add_scaled_sum(x_train, x_train)

In [22]:
x_test = add_scaled_sum(x_train, x_test)
# a = x_test.iloc[:, 784].values
# print(a.min())
# print(a.max())

In [ ]:
# b = x_train.iloc[:, 784].values
# print(b.max())
# print(b.min())

In [23]:
x_test = add_central_sum(x_train, x_test)
# a = x_test.iloc[:, 785].values
# print(a.min())
# print(a.max())
# x_test

In [24]:
x_train = add_central_sum(x_train, x_train)
# b = x_train.iloc[:, 785].values
# print(b.min())
# print(b.max())
# x_train

In [25]:
y_train = y_train.append(y_train, ignore_index=True)
# y_train
from sklearn import linear_model
from sklearn.model_selection import GridSearchCV

In [27]:
x_train.shape

(24000, 786)

In [28]:
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import chi2
select = SelectPercentile(chi2, percentile=60)
x_train = select.fit_transform(x_train, y_train)
x_test = select.transform(x_test)

(24000, 471)

In [ ]:
# from sklearn import linear_model
# from sklearn.model_selection import GridSearchCV
# # # Perceptron
# # percep = linear_model.Perceptron()
# # percep_p_Grid = {'max_iter':np.arange(100, 301, 100), 'penalty':['l2','l1',None]}
# # percepGrid = GridSearchCV(percep, percep_p_Grid)
# # percepGrid.fit(x_train, y_train.values.reshape(-1))
# # print(percepGrid.cv_results_)

In [ ]:
# percepGrid.cv_results_

In [ ]:
np.logspace(-3, 2,11)

In [ ]:
# logistic
logi = linear_model.LogisticRegression(solver = 'liblinear')
log_p_Grid = { 'penalty':['l2','l1'], 'C': np.logspace(-3, 2, 11)}#tried 10 and 100,0.0001, 0.0005, 0.001, 0.005,[0.01, 0.1, 1]
logiGrid = GridSearchCV(logi, log_p_Grid)
logiGrid.fit(x_train, y_train.values.reshape(-1))
logiGrid.cv_results_

In [ ]:
logi = linear_model.LogisticRegression(solver = 'lbfgs')
log_p_Grid = { 'penalty':['l2','none'], 'C': np.logspace(-3, 2, 11), 'max_iter':[200]}#tried 10 and 100,0.0001, 0.0005, 0.001, 0.005,[0.01, 0.1, 1]
logiGrid = GridSearchCV(logi, log_p_Grid)
logiGrid.fit(x_train, y_train.values.reshape(-1))
logiGrid.cv_results_

In [ ]:
# from sklearn.svm import SVC
# # svm
# svc = SVC()
# svc_p_Grid = {'max_iter':[1000]}#'C':[0.01, 0.1, 1, 10], 'kernel':['poly', 'rbf','sigmoid'], 'gamma':[1,.01, .001, .0001, 'scale', 'auto']}
# svcGrid = GridSearchCV(svc, svc_p_Grid)
# svcGrid.fit(x_train, y_train.values.reshape(-1))
# svcGrid.cv_results_

In [29]:
logi = linear_model.LogisticRegression(solver = 'liblinear')
log_p_Grid = { 'penalty':['l2'], 'C': np.logspace(-2, -1, 7), 'max_iter':[200]}#tried 10 and 100,0.0001, 0.0005, 0.001, 0.005,[0.01, 0.1, 1]
logiGrid = GridSearchCV(logi, log_p_Grid)
logiGrid.fit(x_train, y_train.values.reshape(-1))
logiGrid.cv_results_

{'mean_fit_time': array([0.73563185, 0.82339931, 0.88622446, 0.87665625, 0.89879708,
        0.94407582, 1.02565856]),
 'std_fit_time': array([0.03943548, 0.02184611, 0.03482731, 0.01108821, 0.01464872,
        0.03703174, 0.07198349]),
 'mean_score_time': array([0.00578671, 0.00598316, 0.00618324, 0.00618935, 0.00658212,
        0.0071804 , 0.00638299]),
 'std_score_time': array([0.00039974, 0.00089132, 0.00074588, 0.00097551, 0.00135359,
        0.00171505, 0.00135267]),
 'param_C': masked_array(data=[0.01, 0.014677992676220698, 0.021544346900318832,
                    0.03162277660168379, 0.046415888336127774,
                    0.06812920690579612, 0.1],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_iter': masked_array(data=[200, 200, 200, 200, 200, 200, 200],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_penal

In [ ]:
# model = linear_model.LogisticRegression(solver = 'liblinear', max_iter = 200, penalty = 'l2', C = 0.03162277660168379)
model = linear_model.LogisticRegression(solver = 'liblinear', max_iter = 200, penalty = 'l2', C = 0.01)
# model = SVC(max_iter=1000)
model.fit(x_train, y_train.values.reshape(-1))
yproba1_test = model.predict_proba(x_test)[:, 1]
np.savetxt('yproba1_test.txt', yproba1_test)